# Predicting with a trained model on ml4ir

There are at least two ways to get predictions with a model trained on ml4ir. 
1. rely exclusively on TF
2. load the Data+Model with ml4ir 

This notebooks investigates how to do it, assuming a classification model. 

## Way 1: Using TF only

In [15]:
import os, glob
import numpy as np

from tensorflow import data
import tensorflow as tf
from tensorflow.keras import models as kmodels


print(tf.__version__)


MODEL_DIR = "/tmp/org62-verybig-smallmodelwobn-0.005-1024-ones/"

# Notice here: I am usign the tfrecord signature and model version.
# This means that I will hit the model with tfrecords (the model will parse them internally)
# There is the default signature too, that hits the model with tensors, meaning I need to do the preprocessing
model = kmodels.load_model(os.path.join(MODEL_DIR, 'final/tfrecord/'), compile=False)
infer_fn = model.signatures["serving_tfrecord"]



# These are logistics from training. Label mappings that were passed to TF in the feature config
# to get indexes from labels
# key_prefix file used in the feature_config 

key_prefixes = pd.read_csv("/tmp/key_prefix.csv")
id2label = dict(zip(key_prefixes.index, key_prefixes['clicked_key_prefix']))
labeld2id = dict(zip( key_prefixes['clicked_key_prefix'], key_prefixes.index,))

2.0.1


In [16]:
%%time

# Here we are defining our tfrecord dataset. 
# it is an one-liner if the data are in tfrecords format already. 
dataset = data.TFRecordDataset(glob.glob(os.path.join('/tmp', "test_ta", "part*")))
total_preds, total_labels = [], []
i = 0
# My ugly prediction loop, I am using large batches on purpose, since we are predicting
for batch in dataset.batch(1024):
    probs = infer_fn(protos=batch)
    probs = probs['category_label'].numpy()
    probs = probs [:, 0, :] # Squeeze on dim from predictions, because out classification predictions are 3-dim
    total_preds.append(probs)
    
    
    labels = [item.numpy()[-3:].decode("utf-8") for item in batch] # A hachy way I used to get the actual label.
    total_labels += labels
    i +=1 

# Everything happened to populate the following, a labels array (data_size,) and a preds array (data_size, labels_size)
total_labels = np.array(total_labels)
total_preds = np.array([item for sublist in total_preds for item in sublist])
# From this point I can do any analysis I want

# Open questions
# Can I apply any function to the tfrecords on the fly? To avoid creating a new dataset.. 
# Example: instead of using the full query, I want to use only its first 3-chars

CPU times: user 4.06 s, sys: 1.42 s, total: 5.48 s
Wall time: 1.93 s


In [17]:
total_labels.shape, total_preds.shape

((1985,), (1985, 153))

In [71]:
4.06/1985

0.0020453400503778335

## Way 2: using full ml4ir Relevance model, Relevance dataset definition

Above, tfrecords are sent to the model, its `serving_tfrecord` signature. 
Now we will the ml4ir way. Recall, in ml4ir we define a model, a dataset (and other things) and all are brought together in a Relevance Model. 
The Relevance model has `.fit` and `.predict` that we can use on any part (train, validation, test) of the Relevance dataset. 
Doing this, has an advantage: it provides more flexibility that we can use in certain cases.

In [72]:
import json
import yaml
import logging
import pandas as pd
import os
import tensorflow as tf
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.features.feature_config import FeatureConfig, ExampleFeatureConfig
from ml4ir.base.config.keys import TFRecordTypeKey
from ml4ir.applications.classification.model.losses import categorical_cross_entropy
from ml4ir.base.model.scoring.interaction_model import InteractionModel, UnivariateInteractionModel
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.model.scoring.scoring_model import ScorerBase, RelevanceScorer
from ml4ir.applications.ranking.model.metrics import metric_factory
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.config.keys import OptimizerKey
from ml4ir.base.model.optimizer import get_optimizer
from tensorflow.keras.optimizers import Optimizer
from ml4ir.base.config.keys import DataFormatKey
from ml4ir.base.data.relevance_dataset import RelevanceDataset
from ml4ir.base.io.local_io import LocalIO, FileIO

In [74]:
# Setup logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
tf.get_logger().setLevel('INFO')

# Define FileIO, can be LocalIO or SparkIO. Here we are working locally, no need for HDFS
file_io: FileIO = LocalIO(logger)



In [118]:
# Let's define out feature config (the same we used for training)
feature_config = FeatureConfig.get_instance(
    tfrecord_type=TFRecordTypeKey.EXAMPLE,
    feature_config_dict=yaml.safe_load(open("/Users/gbalikas/PycharmProjects/MLConfigs/ml4ir/entity_prediction/configs/feature_config.yaml")),
    logger=logger)

In [81]:
# Define interaction model
interaction_model = UnivariateInteractionModel(
    feature_config=feature_config,
    feature_layer_keys_to_fns={},
    tfrecord_type=TFRecordTypeKey.EXAMPLE,
    file_io=file_io,
)

# Define loss object from loss key
loss = categorical_cross_entropy.get_loss(loss_key='categorical_cross_entropy')

# Define scorer
scorer = RelevanceScorer.from_model_config_file(
    model_config_file="/Users/gbalikas/PycharmProjects/MLConfigs/model_config.yaml",
    feature_config=feature_config,
    interaction_model=interaction_model,
    loss=loss,
    output_name='category_label',
    logger=logger,
    file_io=file_io,
)

# Define metrics objects from metrics keys
metrics= [
    metric_factory.get_metric(metric_key=metric_key) for metric_key in ['categorical_accuracy']
]

# Define optimizer
optimizer: Optimizer = get_optimizer(
    optimizer_key='adam',
    learning_rate=0.01,
    learning_rate_decay=1,
    learning_rate_decay_steps=100,
)


In [82]:
# define relevance model
relevance_model = RelevanceModel(
        feature_config=feature_config,
        scorer=scorer,
        metrics=metrics,
        optimizer=optimizer,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        output_name="category_label", # this needs to be the one we used when training the model
        file_io=file_io,
        logger=logger, 
        model_file="/tmp/org62-verybig-smallmodelwobn-0.005-1024-ones/checkpoint.tf/",
    compile_keras_model=False
    )


Retraining is not yet supported. Model is loaded with compile=False


In [83]:
# Define relevance Dataset
relevance_dataset = RelevanceDataset(
        data_dir="/tmp/dataset", # I created a dir `/tmp/dataset`, with `train`, `validation` being empty 
        data_format=DataFormatKey.TFRECORD,
        feature_config=feature_config,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        batch_size=1024,
        preprocessing_keys_to_fns={},
        file_io=file_io, use_part_files=True,
        logger=logger
    )


{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string', default_value=''), 'organization_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'smart_scope_order': FixedLenFeature(shape=[], dtype='string', default_value='')}
{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string', default_value=''), 'organization_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'smart_scope_order': FixedLenFeature(shape=[], dtype='string', default_value='')}
{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string',

In [84]:
%%time
preds = relevance_model.predict(relevance_dataset.test,inference_signature='serving_default')

CPU times: user 7min 24s, sys: 1min 48s, total: 9min 12s
Wall time: 8min 39s


In [89]:
# lets take a look at our predictions
preds.head(2)
# there is some noise due to padding and tensors, but we can work with this

prediction_id clicked_key_prefix       query  \
0  b'-10jd4gnxu07sq'  b'500'             b'7276897'   
1  b'-112ai2rvrrj89'  b'aVu'             b'bcdata'    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               query_words  \
0  (tf.Tensor(b'7276897', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string))   
1  (tf.Tensor(b'bc', shape=(), dtype=string), tf.Tensor(b'data', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string))    

      organization_id  \
0  b'00D000000000062'   
1  b'00D000000000062'   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              smart_scope_order  \
0  (tf.Tensor(b'005', shape=(), dtype=string), tf.Tensor(b'500', shape=(), dtype=string), tf.Tensor(b'003', shape=(), dtype=string), tf.Tensor(b'ka#', shape=(), dtype=string), tf.Tensor(b'00O', shape=(), dtype=string), tf.Tensor(b'a1h', shape=(), dtype=string), tf.Tensor(b'02u', shape=(), dtype=string), tf.Tensor(b'087', shape=(), dtype=string), tf.Tensor(b'00h', shape=(), dtype=string), tf.Tensor(b'0PS', shape=(), dtype=string), tf.Tensor(b'aCH', sha

In [97]:
def clean_predictions(probabilities):
    return [class_prob.numpy() for class_prob in probabilities]

In [99]:
preds['category_label'] = preds['category_label'].apply(clean_predictions)

In [102]:
preds.head(2)
# we can go further and process anything we want

,prediction_id,clicked_key_prefix,query,query_words,organization_id,smart_scope_order,category_label
0,b'-10jd4gnxu07sq',b'500',b'7276897',"(tf.Tensor(b'7276897', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string))",b'00D000000000062',"(tf.Tensor(b'005', shape=(), dtype=string), tf.Tensor(b'500', shape=(), dtype=string), tf.Tensor(b'003', shape=(), dtype=string), tf.Tensor(b'ka#', shape=(), dtype=string), tf.Tensor(b'00O', shape=(), dtype=string), tf.Tensor(b'a1h', shape=(), dtype=string), tf.Tensor(b'02u', shape=(), dtype=string), tf.Tensor(b'087', shape=(), dtype=string), tf.Tensor(b'00h', shape=(), dtype=string), tf.Tensor(b'0PS', shape=(), dtype=string), tf.Tensor(b'aCH', shape=(), dtype=string), tf.Tensor(b'069', shape=(), dtype=string), tf.Tensor(b'0XC', shape=(), dtype=string), tf.Tensor(b'0F9', shape=(), dtype=string), tf.Tensor(b'066', shape=(), dtype=string), tf.Tensor(b'1Y3', shape=(), dtype=string), tf.Tensor(b'570', shape=(), dtype=string), tf.Tensor(b'00B', shape=(), dtype=string), tf.Tensor(b'00Q', shape=(), dtype=string), tf.Tensor(b'00U', shape=(), dtype=string))","[0.99960655, 5.9950075e-08, 0.0003809429, 1.2930159e-13, 2.5508744e-09, 2.5188523e-10, 1.0457474e-05, 1.4054472e-20, 6.1807084e-16, 5.613605e-21, 1.7621737e-20, 1.1596228e-20, 2.592608e-12, 3.1163613e-20, 1.2295745e-15, 5.0482523e-20, 1.9953651e-20, 1.2083234e-25, 7.857907e-20, 1.8853005e-20, 3.7720486e-20, 2.0098994e-12, 1.1906604e-06, 3.1496368e-18, 4.1222506e-17, 1.0780542e-20, 1.0634542e-31, 5.3388733e-21, 3.009256e-20, 4.904398e-07, 1.7494373e-24, 1.1667623e-20, 2.6727887e-24, 1.4577255e-20, 6.4087183e-27, 1.0694527e-20, 9.351022e-20, 2.344431e-07, 2.7991737e-20, 1.3077041e-20, 4.125249e-20, 5.7446925e-20, 1.04267215e-20, 1.7593056e-20, 6.803406e-21, 8.68809e-21, 4.0961292e-20, 2.159777e-20, 1.3935741e-20, 7.246997e-21, 3.746351e-20, 1.0014895e-20, 5.894689e-21, 2.6589231e-20, 1.5944283e-20, 1.7566901e-20, 1.0154617e-20, 1.7122432e-20, 2.0366372e-21, 3.7785566e-21, 3.008521e-20, 1.0183711e-20, 5.4511138e-21, 1.7097106e-20, 8.0996726e-21, 5.7044867e-21, 3.2272044e-20, 2.351025e-20, 1.1476266e-20, 7.7318284e-21, 4.387241e-25, 7.211422e-21, 3.49494e-21, 3.4722577e-20, 2.980114e-20, 1.3627363e-20, 1.9528936e-20, 3.2454263e-20, 1.18064636e-20, 8.4376713e-20, 2.1507003e-20, 1.21111375e-20, 1.7415706e-20, 3.3729333e-20, 6.3668257e-21, 2.6579901e-20, 1.377454e-20, 2.1005588e-20, 1.3897941e-20, 2.2128172e-20, 9.546223e-21, 2.8033946e-20, 9.185067e-21, 1.2130743e-20, 7.623349e-21, 1.4398966e-20, 1.2871596e-28, 1.7392999e-20, 2.8308722e-28, 1.0054015e-20, ...]"
1,b'-112ai2rvrrj89',b'aVu',b'bcdata',"(tf.Tensor(b'bc', shape=(), dtype=string), tf.Tensor(b'data', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.

###  Applying custom functions before predicting
Above we saw that the query was sent in full in the model. Reading from tf records, it is not easy to apply data transformations on the fly to call the `.fit` method directly. 
This need can arise in practise, after having a model to evaluate different aspects of it. 
In this example, we want to evaluate on truncated queries (queries that are not full, e.g,. consist of the first n characters.)

To do that we need:
    - a function that implements this logic
    - to apply the function before the data hitting the model
    
This is done as follows:
    - we write the function
    - we inform the pipeline of ist existence and where it must be applied
    - we call `.predict`


In [103]:
# Here is the truncation function
@tf.function
def trim_text(inp, keep_first=3):
    """Keeps the first `keep_first` bytes of a tf.string"""
    return tf.strings.substr(inp, 0, keep_first, unit='BYTE')

s = tf.constant('ml4ir rocks')
trim_text(s, keep_first=5)

<tf.Tensor: id=85776706, shape=(), dtype=string, numpy=b'ml4ir'>

In [104]:
# we need to inform the pipeline. This needs to be done in the feature config. 
# We either 
# (i) go and preprocess the feature_config.yaml file
# or 
# (ii) keep in mind that the file is only a means to use data, we can adapt it once loaded

In [119]:
feature_config.features[0]

{'name': 'query',
 'node_name': 'query',
 'trainable': True,
 'dtype': 'string',
 'log_at_inference': True,
 'feature_layer_info': {'type': 'numeric',
  'shape': None,
  'fn': 'bytes_sequence_to_encoding_bilstm',
  'args': {'encoding_type': 'bilstm',
   'encoding_size': 128,
   'embedding_size': 128,
   'max_length': 20}},
 'preprocessing_info': [{'fn': 'preprocess_text',
   'args': {'remove_punctuation': True, 'to_lower': True}}],
 'serving_info': {'name': 'query', 'required': True},
 'default_value': ''}

In [120]:
feature_config.features[0]['preprocessing_info'].insert(0, {'fn': 'trim_text', 'args':{'keep_first':3}})

In [121]:
feature_config.features[0]

{'name': 'query',
 'node_name': 'query',
 'trainable': True,
 'dtype': 'string',
 'log_at_inference': True,
 'feature_layer_info': {'type': 'numeric',
  'shape': None,
  'fn': 'bytes_sequence_to_encoding_bilstm',
  'args': {'encoding_type': 'bilstm',
   'encoding_size': 128,
   'embedding_size': 128,
   'max_length': 20}},
 'preprocessing_info': [{'fn': 'trim_text', 'args': {'keep_first': 3}},
  {'fn': 'preprocess_text',
   'args': {'remove_punctuation': True, 'to_lower': True}}],
 'serving_info': {'name': 'query', 'required': True},
 'default_value': ''}

In [122]:
feature_config.features[1]['preprocessing_info'].insert(0, {'fn': 'trim_text', 'args':{'keep_first':3}})

In [127]:
# Now at this point the feature_config is adapted for the use-case
# We need to define again the RelevanceDataset and show the implementation of our function

relevance_dataset = RelevanceDataset(
        data_dir="/tmp/dataset", # I created a dir `/tmp/dataset`, with `train`, `validation` being empty 
        data_format=DataFormatKey.TFRECORD,
        feature_config=feature_config,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        batch_size=1024,
        preprocessing_keys_to_fns={'trim_text': trim_text},
        file_io=file_io, use_part_files=True,
        logger=logger
    )


{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string', default_value=''), 'organization_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'smart_scope_order': FixedLenFeature(shape=[], dtype='string', default_value='')}
{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string', default_value=''), 'organization_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'smart_scope_order': FixedLenFeature(shape=[], dtype='string', default_value='')}
{'prediction_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked_key_prefix': FixedLenFeature(shape=[], dtype='string', default_value=''), 'query': FixedLenFeature(shape=[], dtype='string',

In [129]:
# Let's inspect the data that will hit the model
next(iter(relevance_dataset.test))
# Notice how query and query_words are short, 3 bytes long! 

({'prediction_id': <tf.Tensor: id=85777272, shape=(1024, 1), dtype=string, numpy=
  array([[b'-10jd4gnxu07sq'],
         [b'-112ai2rvrrj89'],
         [b'-12s0iuwulu0bd'],
         ...,
         [b'-a8qqrwhpi0gm'],
         [b'-bgk09cn8nfcy'],
         [b'-bqob56px4fq1']], dtype=object)>,
  'query': <tf.Tensor: id=85777273, shape=(1024, 1), dtype=string, numpy=
  array([[b'727'],
         [b'bc'],
         [b'273'],
         ...,
         [b'cha'],
         [b'zan'],
         [b'952']], dtype=object)>,
  'query_words': <tf.Tensor: id=85777274, shape=(1024, 1, 20), dtype=string, numpy=
  array([[[b'727', b'', b'', ..., b'', b'', b'']],
  
         [[b'bc', b'', b'', ..., b'', b'', b'']],
  
         [[b'273', b'', b'', ..., b'', b'', b'']],
  
         ...,
  
         [[b'cha', b'', b'', ..., b'', b'', b'']],
  
         [[b'zan', b'', b'', ..., b'', b'', b'']],
  
         [[b'952', b'', b'', ..., b'', b'', b'']]], dtype=object)>,
  'organization_id': <tf.Tensor: id=85777271, shape=(1

In [131]:
%%time
# Let's get predictions again
preds = relevance_model.predict(relevance_dataset.test,inference_signature='serving_default')

CPU times: user 7min 35s, sys: 1min 51s, total: 9min 27s
Wall time: 8min 59s


In [132]:
# Same cleaning on predictions
preds['category_label'] = preds['category_label'].apply(clean_predictions)

In [134]:
preds.head()
# See  how the query and the query words have been processed!

,prediction_id,clicked_key_prefix,query,query_words,organization_id,smart_scope_order,category_label
0,b'-10jd4gnxu07sq',b'500',b'727',"(tf.Tensor(b'727', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string))",b'00D000000000062',"(tf.Tensor(b'005', shape=(), dtype=string), tf.Tensor(b'500', shape=(), dtype=string), tf.Tensor(b'003', shape=(), dtype=string), tf.Tensor(b'ka#', shape=(), dtype=string), tf.Tensor(b'00O', shape=(), dtype=string), tf.Tensor(b'a1h', shape=(), dtype=string), tf.Tensor(b'02u', shape=(), dtype=string), tf.Tensor(b'087', shape=(), dtype=string), tf.Tensor(b'00h', shape=(), dtype=string), tf.Tensor(b'0PS', shape=(), dtype=string), tf.Tensor(b'aCH', shape=(), dtype=string), tf.Tensor(b'069', shape=(), dtype=string), tf.Tensor(b'0XC', shape=(), dtype=string), tf.Tensor(b'0F9', shape=(), dtype=string), tf.Tensor(b'066', shape=(), dtype=string), tf.Tensor(b'1Y3', shape=(), dtype=string), tf.Tensor(b'570', shape=(), dtype=string), tf.Tensor(b'00B', shape=(), dtype=string), tf.Tensor(b'00Q', shape=(), dtype=string), tf.Tensor(b'00U', shape=(), dtype=string))","[0.9999974, 1.311562e-07, 1.9355932e-06, 4.7308484e-16, 4.6343224e-12, 2.692105e-11, 4.023711e-09, 3.2256282e-24, 1.5762118e-18, 1.0973475e-24, 3.352345e-24, 1.8507192e-24, 3.7755e-18, 6.794071e-24, 1.464972e-17, 1.1941628e-23, 5.102617e-24, 8.731958e-30, 1.3228216e-23, 3.1610798e-24, 9.7018855e-24, 1.1766184e-15, 1.5807933e-08, 5.9237114e-21, 2.9421913e-18, 1.739326e-24, 9.893776e-37, 9.683577e-25, 5.674667e-24, 1.06721814e-07, 1.3729087e-27, 2.4159116e-24, 1.789087e-28, 2.6134193e-24, 3.2131377e-32, 1.8990144e-24, 2.0950377e-23, 8.645745e-08, 4.231009e-24, 2.224267e-24, 1.0019675e-23, 1.5671187e-23, 1.9138501e-24, 5.3215105e-24, 2.4020172e-24, 1.7201775e-24, 1.6586783e-22, 3.7300653e-24, 3.2184992e-24, 1.9393964e-24, 6.890961e-24, 1.9295376e-24, 1.3749022e-24, 4.7011633e-24, 2.8305378e-24, 4.267627e-24, 2.6757008e-24, 3.3504015e-24, 6.8799256e-25, 4.965094e-25, 5.6283593e-24, 2.3708471e-24, 1.0974521e-24, 2.804378e-24, 2.3255408e-24, 1.251605e-24, 5.5661193e-24, 4.497636e-24, 2.5492594e-24, 1.5655465e-24, 4.6998758e-30, 1.1732964e-24, 6.3390515e-25, 4.833199e-24, 5.4776095e-24, 3.3819646e-24, 4.61576e-24, 5.6922503e-24, 2.3476368e-24, 1.4934454e-23, 3.5271678e-23, 2.07006e-24, 6.045539e-24, 6.4445077e-24, 1.1752719e-24, 3.0354384e-24, 2.4866556e-24, 3.255581e-24, 2.2685447e-23, 4.2077987e-24, 2.4756102e-24, 6.585768e-24, 1.4655424e-24, 2.6004117e-24, 2.9116233e-24, 2.0340059e-24, 6.2395516e-35, 4.6963063e-24, 1.1221922e-32, 2.4434854e-24, ...]"
1,b'-112ai2rvrrj89',b'aVu',b'bc',"(tf.Tensor(b'bc', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dtype=string), tf.Tensor(b'', shape=(), dty